<a href="https://colab.research.google.com/github/punkmic/Topic-Modeling-Reclame-Aqui/blob/master/Topic_Modeling_with_BERTopic_Reclame_aqui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Topic Modeling with BERTopic - Reclame Aqui**

BERTopic is a topic modeling technique that leverages 🤗 transformers and a custom class-based TF-IDF to create dense clusters allowing for easily interpretable topics whilst keeping important words in the topic descriptions (https://maartengr.github.io/BERTopic/index.html).

## **Import dependecies**

In [ ]:
import pandas as pd # for data manipulation
import os # for interacting with the operating system
import nltk # for natural language processing
import string # for string manipulation
import re # for for regular expressions
import matplotlib.pyplot as plt # for visualization
import spacy # for lemmatize portuguese text
try:
  from bertopic import BERTopic # for topic modeling
  from spellchecker import SpellChecker # for spell check
  from sentence_transformers import SentenceTransformer # for embeddings
  from umap import UMAP # for dimension reduction
  from umap.evaluation import optimal_umap
  from hdbscan import HDBSCAN # for clustering
except:
  !pip install pyspellchecker
  !pip install sentence-transformers
  !pip install umap-learn
  !pip install hdbscan
  !pip install bertopic
  #os.kill(os.getpid(), 9)

In [2]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [69]:
# Donwload dependency need to stem portuguese text
nltk.download('rslp')

[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.


True

## **Load data from [Github](https://github.com/punkmic/Topic-Modeling-Reclame-Aqui.git)**

In [6]:
!git clone https://github.com/punkmic/Topic-Modeling-Reclame-Aqui.git

fatal: destination path 'Topic-Modeling-Reclame-Aqui' already exists and is not an empty directory.


## **Run this cell to update files from remote repository**

In [7]:
# Change directory
%cd /content/Topic-Modeling-Reclame-Aqui 

# Update files from remote repository
!git pull 

# Return to work directory
%cd ..

# Check current directory
!pwd

/content/Topic-Modeling-Reclame-Aqui
Already up to date.
/content
/content


## **Read csv file using read_csv() method as Dataframe**

In [8]:
# Set the path to data
path_csv = "/content/Topic-Modeling-Reclame-Aqui/corpus.csv"

# Read the CSV file using the read_csv method
df = pd.read_csv(path_csv)

# Print the first 5 rows of the DataFrame
df.head(5)

,title,text
0,Pedido Cancelado sem justificativa após uma se...,Eu estava pesquisando bastante uma nova TV par...
1,Pedido cancelado,Eu sinceramente estou decepcionada com o Amazo...
2,Cobrança indevida,Cancelei meu plano antes de terminar o período...
3,PEDIDO REINCIDENTE,"OLHA FIZ COMPRA VEIO ERRADA, E VEIO ERRADO NOV..."
4,Assinatura para vender na amazon Brasil,Eu me inscrevi na Amazon para realizar vendas ...


This dataset contains just two columns called title and text 

In [9]:
print(df.shape)

(12760, 2)


There are 12760 rows in this dataset. To make sure that we have unique rows let's remove duplicate rows.

In [10]:
print(f"Shape before remove duplicates: {df.shape}")

# User the drop_duplicated method to drop duplicates rows
df = df.drop_duplicates(subset="text")

print(f"Shape after remove duplicates: {df.shape}")

Shape before remove duplicates: (12760, 2)
Shape after remove duplicates: (10510, 2)


### **Now let's lower each element in the Dataframe, remove unwanted text and join the two columns**

In [11]:
# apply the str.lower() method to each element in the dataframe
df = df.applymap(str.lower)

# Use the replace() method to replace the string with an empty string
df = df.replace(re.compile('\[editado pelo reclame aqui\]|editado pelo reclame aqui|Editado pelo Reclame Aqui'), '')

# join columns
df["documents"] = df["title"] + " " + df["text"]

# Drop the old index column
df.reset_index(inplace = True, drop = True)

df.head()

,title,text,documents
0,pedido cancelado sem justificativa após uma se...,eu estava pesquisando bastante uma nova tv par...,pedido cancelado sem justificativa após uma se...
1,pedido cancelado,eu sinceramente estou decepcionada com o amazo...,pedido cancelado eu sinceramente estou decepc...
2,cobrança indevida,cancelei meu plano antes de terminar o período...,cobrança indevida cancelei meu plano antes de ...
3,pedido reincidente,"olha fiz compra veio errada, e veio errado nov...",pedido reincidente olha fiz compra veio errada...
4,assinatura para vender na amazon brasil,eu me inscrevi na amazon para realizar vendas ...,assinatura para vender na amazon brasil eu me ...


## **Preprocessing**

### **Tokenization**

Tokenization aims to breaking text down into its component parts

In [67]:
WORD_TOKENIZER = nltk.tokenize.word_tokenize
def tokenize(text, lowercase=True):
  if lowercase:
    text = text.lower()
  return WORD_TOKENIZER(text, language="portuguese")

### **Stem** 

Stem the tokens. This step aims to remove morphological affixes and normalize to standardized stem forms

In [70]:
STEMMER = nltk.stem.RSLPStemmer()
def stem(tokens):
  return [STEMMER.stem(token) for token in tokens]

### **Lemmatize**

Lemmatize the tokens. Retains more natural forms than stemming, but assumes all tokens nons unless tokens are passed as (word, pos) tuples. Note: nltk lemmatize does not suport portugues language

In [61]:
LEMMATIZER = nltk.WordNetLemmatizer()

def lemmatize(tokens):
  lemmas = []
  for token in tokens:
      if isinstance(token, str):
          # treats token like a noun
          lemmas.append(LEMMATIZER.lemmatize(token)) 
      else: 
          # assume a tuple of (word, pos)
          lemmas.append(LEMMATIZER.lemmatize(*token))
  return lemmas

### **Remove stopwords**

Stop words are things like articles and conjunctions that usually do not offer a lot of value in an analysis.

In [71]:
def remove_stopwords(tokens, stopwords=None):

  # Use the default stop words if none is passed
  if stopwords is None:
    stopwords = nltk.corpus.stopwords.words('portuguese')
  
  # Filter the list of tokens to exclude the stop word tokens
  return [token for token in tokens if token not in stopwords]

### **Remove hyperlinks**

Removes http/s links from the tokens.

In [16]:
def remove_links(tokens):
  # Filter tokens that starts with "http://" or "https://"
  return [token for token in tokens 
          if not token.startswith("http://")
          and not token.startswith("https://")]

### **Remove numbers**

In [72]:
def remove_numbers(tokens):
  # Filter out number tokens using a list comprehension and the isnumeric method
  return [token for token in tokens if not token.isnumeric()]


### **Remove date**

In [18]:
def remove_date(tokens):
  # Compile a regular expression to match dates in the format dd/mm or dd/mm/yyyy
  date_regex = re.compile(r'\d{2}/\d{2}(/\d{4})?')

  # Use the regex to find all the tokens that match the date pattern
  dates = [token for token in tokens if date_regex.fullmatch(token)]

  # Filter the list of tokens to exclude the date tokens
  filtered_tokens = [token for token in tokens if token not in dates]

  # Return the filtered tokens
  return filtered_tokens

### **Remove punctuation**

In [19]:
def remove_punctuation(tokens,
                       strip_mentions=False,
                       strip_hashtags=False,
                       strict=False):

    # Use the sub method to remove all punctuation characters
    tokens = [re.sub(r"[,!?.]", "", t) for t in tokens ] 

    # Remove punctuation
    #tokens = [t for t in tokens if t not in string.punctuation]

    # Remove @ symbol from left side of tokens
    if strip_mentions:
        tokens = [t.lstrip('@') for t in tokens]

    # Remove # symbol from left side of tokens
    if strip_hashtags:
        tokens = [t.lstrip('#') for t in tokens]

    return tokens

### **Remove short tokens**

In [20]:
def remove_short_tokens(tokens):
  # Filter the list of tokens to exclude tokens that are shorter than four letters
  filtered_tokens = [token for token in tokens if len(token) >= 4]

  # Return the filtered tokens
  return filtered_tokens

### **Correction of spelling errors**

In [21]:
def check_spell_errors(text):
  # Create a SpellChecker object
  spell = SpellChecker(language='pt')

  # Correct the spelling errors in the text
  corrected_text = spell.correction(text)

  # If no correction is present user the original text
  if corrected_text == None:
     corrected_text =  text
  
  # Return the corrected text
  return corrected_text

In [76]:
def preprocessing(documents):
  corpus = []

  # process each document and append to corpus list
  for i, text in enumerate(documents):
    if i % 1000 == 0:
      print(f"{i} documents of {len(documents)}")
    text = check_spell_errors(text)
    tokens = tokenize(text)
    tokens = remove_links(tokens)
    tokens = remove_punctuation(tokens, strip_mentions=True, strip_hashtags=True)
    tokens = remove_numbers(tokens)
    tokens = remove_date(tokens)
    tokens = remove_short_tokens(tokens)
    tokens = remove_stopwords(tokens)
    tokens = stem(tokens) 
    corpus.append(' '.join(tokens))
  return corpus

In [ ]:
# Print the first document before and after pre-processing it
corpus = preprocessing(df.documents) 
print(df.documents[0])
print()
corpus[0]

0 documents of 10510
1000 documents of 10510


## **Training a BERTopic Model**

The BERTopic algorithm has several advantages over other topic modeling algorithms. It is able to handle sparse data, it is scalable to large datasets, and it is able to learn topics that are not well-defined or are overlapping.

As our data language is portuguese we will going to set language to multilingual.

### **Enabling the GPU**

We will use the GPU provided by COLAB to accelarate our model training. To enable GPUs for the notebook:
1- Navigate to Edit -> Notebook Settings
2- Select GPU from the Hardware Accelerator drop-down

In [24]:
# verify if GPU is enable
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Not connected to a GPU


Create a new BERTopic model and train it. By default BERTopic use the paraphrase-multilingual-MiniLM-L12-v2 model for multi language documents. For others model check here [BERTopic sentence transformers](https://maartengr.github.io/BERTopic/getting_started/embeddings/embeddings.html#sentence-transformers)

In [42]:
# Create a new BERTopic model using multilingual option
topic_model = BERTopic(language="multilingual", calculate_probabilities=True, verbose=True)

# Train model 
topics, probs = topic_model.fit_transform(corpus)

Batches:   0%|          | 0/329 [00:00<?, ?it/s]

2022-12-11 03:43:54,636 - BERTopic - Transformed documents to Embeddings
2022-12-11 03:44:10,984 - BERTopic - Reduced dimensionality
2022-12-11 03:44:14,685 - BERTopic - Clustered reduced embeddings


BERTopic works in three main steps: 


1.   Documents are first converted to numeric data. It extracts different embeddings based on the context of the word. For this, a sentence transformation model is used.
2.  Documents with similar topics are then grouped together forming clusters with similar topics. For this purpose, BERTopic uses the clustering algorithm UMAP to lower the dimensionality of the embeddings. Then the documents are clustered with the density-based algorithm HDBSCAN.
3. BERTopic extract topics from clusters using a class-based TF-IDF score. This score gives the importance of each word in cluster. Topic are then created based on most important words measured by its C-TF-IDF score.

For more information check this link [BERTopic](https://towardsdatascience.com/topic-modeling-with-bert-779f7db187e6)



## **Extracting Topics**

In [43]:
# print the most frequent topics
freq = topic_model.get_topic_info()
freq.head(5)

,Topic,Count,Name
0,-1,4990,-1_produto_compra_pedido_entrega
1,0,724,0_casas_bahia_whatsapp_entrega
2,1,345,1_celular_samsung_aparelho_comprei
3,2,313,2_endereço_pedido_entregue_shein
4,3,270,3_cancelamento_cancelar_pedido_cancelado


The table above shows the five most freqeuente topics and the words present on it extract by BERTopic. -1 refers to all outliers and should be ignored.

In [44]:
# show the most frequent topic
topic_model.get_topic(1)

[('celular', 0.035869695068834),
 ('samsung', 0.029649884066389198),
 ('aparelho', 0.019810755291385734),
 ('comprei', 0.012916737984595703),
 ('smartphone', 0.012110300496881584),
 ('galaxy', 0.011892953898938784),
 ('produto', 0.011798878728084822),
 ('americana', 0.010137968464132147),
 ('defeito', 0.01001326182850664),
 ('loja', 0.00997724042300456)]

**Note:** BERTopic is stocastich which means that the topics might differ across runs this is mostly due to the stocastisch nature of UMAP

## **Visualization**

### **Intertopic Distance Map**

This graph shows the distance intertopic and help us understand the promixity of topics

In [45]:
topic_model.visualize_topics()

### **Visualize Topic Hierarchy**

The topics that were created can be hierarchically reduced. This visualization shows how the topics relate to one another.

In [46]:
topic_model.visualize_hierarchy(top_n_topics=50)

### **Visualize Terms**

We can visualize the selected terms for a few topics by creating bar charts out of the c-TF-IDF scores for each topic representation.

In [53]:
topic_model.visualize_barchart(top_n_topics=24)

### **Visualize Topic Similarity**

This plot shows a similarity matrix by simply applying cosine similarities through those topic embeddings generate by BERTopic through both c-TF-IDF and embeddings. This matrix indicate how similar certain topics are to each other.

In [48]:
topic_model.visualize_heatmap(n_clusters=20, width=1000, height=1000)

### **Visualize Term Score Decline**

Topics are represented by a number of words starting with the best representative word. Each word is represented by a c-TF-IDF score. The higher the score, the more representative a word to the topic is. Since the topic words are sorted by their c-TF-IDF score, the scores slowly decline with each word that is added.

In [49]:
topic_model.visualize_term_rank()

## **Term search**

In [59]:
# Find topics that contains blackfriday term
similar_topics, similarity = topic_model.find_topics("blackfriday", top_n=5)

# Show similar topics
similar_topics

[65, 23, 68, 64, 20]

In [60]:
# Show a specific topic
topic_model.get_topic(20)

[('curitiba', 0.21259335087648767),
 ('parado', 0.0522626897800864),
 ('trânsito', 0.051179972427955314),
 ('encomenda', 0.04193574303534422),
 ('parada', 0.03581736973479072),
 ('pedido', 0.03351346729632609),
 ('shein', 0.02820351870096115),
 ('novembro', 0.027765812432384497),
 ('chegou', 0.025759335897538982),
 ('dia', 0.023678144634788456)]

### **Topic Reduction**

In [41]:
#new_topic_model = topic_model.reduce_topics(df.documents, nr_topics=40)
#new_topics = new_topic_model.topics_
#new_probs = new_topic_model.probabilities_

2022-12-11 03:28:29,843 - BERTopic - Reduced number of topics from 13 to 13


IndexError: ignored

In [36]:
#new_topic_model.visualize_topics()